Ich habe im Notebook "Gewinnsteuer zusammenfügen" das Problem gehabt, dass ab 2010 mein vorheriger Code nicht mehr funktioniert, weil die Kolonne drei zwei Zahlen beinhaltet und durch /n getrennt ist. Dieses Problem versuche ich hier zu lösen.

In [140]:
import camelot
import tabula
import pandas as pd

In [141]:
Firmen_2010 = camelot.read_pdf("Firmen_2010.pdf", flavor='stream', pages='all', table_areas=['0,842,595,0'])

In [142]:
#zurechtschneiden
test_df = Firmen_2010[0].df
Firmen_10 = test_df[12:]
firmen_2010 = Firmen_10.head(5)
firmen_2010

,0,1,2,3,4,5
12,,0,10'000,21'101\n \n1'051'519,6'782,116'360
13,,10'100,50'000,4'270\n \n8'379'718,279,427'696
14,,50'100,250'000,3'302\n \n35'558'173,77,448'731
15,,250'100,+,2'028\n \n388'354'588,34,1'503'531
16,,Total,,30'701\n \n433'343'998,7'172,2'496'318


In [143]:

#headers setzen
headers = ['Weglassen', 'Steuerbarer Gewinn ab CHF', 'weglassen', 'etwas seltsames',
           'lassweg', 'überflüssig']
firmen_2010.columns = headers
firmen_2010

,Weglassen,Steuerbarer Gewinn ab CHF,weglassen,etwas seltsames,lassweg,überflüssig
12,,0,10'000,21'101\n \n1'051'519,6'782,116'360
13,,10'100,50'000,4'270\n \n8'379'718,279,427'696
14,,50'100,250'000,3'302\n \n35'558'173,77,448'731
15,,250'100,+,2'028\n \n388'354'588,34,1'503'531
16,,Total,,30'701\n \n433'343'998,7'172,2'496'318


In [144]:
# firmen_2010['A'], firmen_2010['B'] = firmen_2010['etwas seltsames'].str.split('\').str
# von https://stackoverflow.com/questions/14745022/how-to-split-a-column-into-two-columns
# gibt errormeldung

### So erhalte ich (nach 2 Stunden ausprobieren) aus der einen Kolonne zwei Kolonnen

In [145]:
getrennte_kolonnen = firmen_2010['etwas seltsames'].str.split(' ', 1, expand=True) 
# von https://stackoverflow.com/questions/14745022/how-to-split-a-column-into-two-columns
getrennte_kolonnen

,0,1
12,21'101\n,\n1'051'519
13,4'270\n,\n8'379'718
14,3'302\n,\n35'558'173
15,2'028\n,\n388'354'588
16,30'701\n,\n433'343'998


In [146]:
#firmen_2010['Anzahl Steuerpflichtige 2010'], firmen_2010['Betrag Kantonssteuer 2010 in CHF'] = firmen_2010['etwas seltsames'].str.split(' ', 1, expand=True) 
#fügt mir nur zwei kolonnen hinten an

In [147]:
firmen_2010

,Weglassen,Steuerbarer Gewinn ab CHF,weglassen,etwas seltsames,lassweg,überflüssig
12,,0,10'000,21'101\n \n1'051'519,6'782,116'360
13,,10'100,50'000,4'270\n \n8'379'718,279,427'696
14,,50'100,250'000,3'302\n \n35'558'173,77,448'731
15,,250'100,+,2'028\n \n388'354'588,34,1'503'531
16,,Total,,30'701\n \n433'343'998,7'172,2'496'318


### Ich versuche, firmen_2010 mit getrennte_kolonnen zu mergen

In [148]:
del firmen_2010['Weglassen']
del firmen_2010['weglassen']
del firmen_2010['lassweg']
del firmen_2010['etwas seltsames']

In [149]:
firmen_2010

,Steuerbarer Gewinn ab CHF,überflüssig
12,0,116'360
13,10'100,427'696
14,50'100,448'731
15,250'100,1'503'531
16,Total,2'496'318


In [150]:
del firmen_2010['überflüssig']

In [151]:
firmen_2010

,Steuerbarer Gewinn ab CHF
12,0
13,10'100
14,50'100
15,250'100
16,Total


In [152]:
#pd.merge(dfunfälle,einwohnerzahlenneu,
        # how ='left',on='Gemeinde')

In [153]:
pd.merge(firmen_2010, getrennte_kolonnen, 
         how = right, left_index = False)  #, left_index=False, right_index=False

NameError: name 'right' is not defined

## das funktioniert nicht, weil ich keine gemeinsame kolonne habe der beiden. ich überlege mir ernsthaft, eine neue kolonne hinzuzufügen. so wie vorhin. 

In [ ]:
firmen_2010['alibi kolonne'] = firmen_2010['Steuerbarer Gewinn ab CHF'].str.split(' ', 1, expand=True) 

In [ ]:
firmen_2010

In [154]:
getrennte_kolonnen['alibi kolonne'] = getrennte_kolonnen['Steuerbarer Gewinn ab CHF'].str.split(' ', 1, expand=True) 

KeyError: 'Steuerbarer Gewinn ab CHF'

### funktioniert nicht. 

In [155]:
firmen_2010['Steuerbarer Gewinn ab CHF'] = pd.Series(getrennte_kolonnen.random.randn(sLength), index=firmen_2010.index)

AttributeError: 'DataFrame' object has no attribute 'random'

In [156]:
getrennte_kolonnen['Steuerbarer Gewinn ab CHF'] = firmen_2010.values

## Alles Arschlöcher!!!!!!!!!!!!!!!!!! Nochmals von vorne mit diesen beiden Dataframes hier

In [157]:
firmen_2010

,Steuerbarer Gewinn ab CHF
12,0
13,10'100
14,50'100
15,250'100
16,Total


In [125]:
getrennte_kolonnen

,0,1
12,21'101\n,\n1'051'519
13,4'270\n,\n8'379'718
14,3'302\n,\n35'558'173
15,2'028\n,\n388'354'588
16,30'701\n,\n433'343'998


In [163]:
#pd.merge(firmen_2010, getrennte_kolonnen,
        #how ='left', on = "Index")
    
#das hatte nicht funktioniert

## Versuch mit Concat
defunden hier dank dem Hinweis auf zip in einem älteren Notebook https://stackoverflow.com/questions/34318141/zip-pandas-dataframes-into-a-new-dataframe

In [164]:
Firmen_2010_neu = pd.concat([firmen_2010, getrennte_kolonnen], axis=1)
Firmen_2010_neu

,Steuerbarer Gewinn ab CHF,0,1,Steuerbarer Gewinn ab CHF
12,0,21'101\n,\n1'051'519,0
13,10'100,4'270\n,\n8'379'718,10'100
14,50'100,3'302\n,\n35'558'173,50'100
15,250'100,2'028\n,\n388'354'588,250'100
16,Total,30'701\n,\n433'343'998,Total


In [165]:
#headers setzen
headers = ['Steuerbarer Gewinn ab CHF', 'Anzahl Steuerpflichtige 2010', 'Betrag Kantonssteuer 2010 in CHF',
           'streichen']
Firmen_2010_neu.columns = headers
Firmen_2010_neu

,Steuerbarer Gewinn ab CHF,Anzahl Steuerpflichtige 2010,Betrag Kantonssteuer 2010 in CHF,streichen
12,0,21'101\n,\n1'051'519,0
13,10'100,4'270\n,\n8'379'718,10'100
14,50'100,3'302\n,\n35'558'173,50'100
15,250'100,2'028\n,\n388'354'588,250'100
16,Total,30'701\n,\n433'343'998,Total


In [166]:
del Firmen_2010_neu['streichen']
Firmen_2010_neu

,Steuerbarer Gewinn ab CHF,Anzahl Steuerpflichtige 2010,Betrag Kantonssteuer 2010 in CHF
12,0,21'101\n,\n1'051'519
13,10'100,4'270\n,\n8'379'718
14,50'100,3'302\n,\n35'558'173
15,250'100,2'028\n,\n388'354'588
16,Total,30'701\n,\n433'343'998


Jetzt muss ich putzen, und zwar etwas mehr als vorhin

In [167]:
def geputzt(zeile):
    zeile = zeile.replace('(', '')
    zeile = zeile.replace(')', '')
    zeile = zeile.replace('\n', '') #das neu rein
    zeile = zeile.replace("'", "").strip()
    return zeile
Firmen_2010_neu['Steuerbarer Gewinn ab CHF'] = Firmen_2010_neu['Steuerbarer Gewinn ab CHF'].apply(geputzt)
Firmen_2010_neu['Anzahl Steuerpflichtige 2010'] = Firmen_2010_neu['Anzahl Steuerpflichtige 2010'].apply(geputzt)
Firmen_2010_neu['Betrag Kantonssteuer 2010 in CHF'] = Firmen_2010_neu['Betrag Kantonssteuer 2010 in CHF'].apply(geputzt)

In [168]:
Firmen_2010_neu

,Steuerbarer Gewinn ab CHF,Anzahl Steuerpflichtige 2010,Betrag Kantonssteuer 2010 in CHF
12,0,21101,1051519
13,10100,4270,8379718
14,50100,3302,35558173
15,250100,2028,388354588
16,Total,30701,433343998


# Juhuiii, ich hab die Chose gepackt! Nach fast einem Tag Arbeit!

### morgen schaue ich, wie ich von hier weiter gehe. Achtung: Den Code oben werde ich voraussichtlich auch bei den Einkommenssteuern brauchen können. Dort hab ich dieselbe Chose gesichtet

In [169]:
Firmen_2010_neu.T

,12,13,14,15,16
Steuerbarer Gewinn ab CHF,0,10100,50100,250100,Total
Anzahl Steuerpflichtige 2010,21101,4270,3302,2028,30701
Betrag Kantonssteuer 2010 in CHF,1051519,8379718,35558173,388354588,433343998


Hier dann schauen, ob das gleich ist wie bei den Vorherigen, oder ob ich etwas kleines anders machen muss..

Hier der ganze bisherige Code, der aber noch ergänzt werden muss:

In [ ]:
#zurechtschneiden
test_df = Firmen_2010[0].df
Firmen_10 = test_df[12:]
firmen_2010 = Firmen_10.head(5)

#headers setzen
headers = ['Weglassen', 'Steuerbarer Gewinn ab CHF', 'weglassen', 'Anzahl Steuerpflichtige 2010', 'Betrag Kantonssteuer 2010 in CHF', 
           'lassweg']
firmen_2010.columns = headers

#überflüssige kolonnen weg
del firmen_2010['Weglassen']
del firmen_2010['weglassen']
del firmen_2010['lassweg']

#putzen
def geputzt(zeile):
    zeile = zeile.replace('(', '')
    zeile = zeile.replace(')', '')
    zeile = zeile.replace("'", "").strip()
    return zeile
firmen_2010['Steuerbarer Gewinn ab CHF'] = firmen_2010['Steuerbarer Gewinn ab CHF'].apply(geputzt)
firmen_2010['Anzahl Steuerpflichtige 2010'] = firmen_2010['Anzahl Steuerpflichtige 2010'].apply(geputzt)
firmen_2010['Betrag Kantonssteuer 2010 in CHF'] = firmen_2010['Betrag Kantonssteuer 2010 in CHF'].apply(geputzt)

#um 90 Grad drehen
firmen_10 = firmen_2010.T

#headers weg
Firmen10 = firmen_10[1:]

#headers neu setzen
headers = ['Steuerbarer Gewinn 0-10000 CHF', 'Steuerbarer Gewinn 10100-50000 CHF', 'Steuerbarer Gewinn 50100-250000 CHF', 
           'Steuerbarer Gewinn ab 250100 CHF', 'Total Steuerbarer Gewinn']
Firmen10.columns = headers

#nur den Betrag Kantonssteuer rausziehen
Betrag_Kantonssteuer_2010 = Firmen10.drop(['Anzahl Steuerpflichtige 2010'])
Betrag_Kantonssteuer_2010